In [1]:
import numpy as np
import pandas as pd
import json
from tqdm import tqdm

from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.callbacks import EarlyStopping
from keras.optimizers import Adam

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

import statsmodels.api as sm

In [2]:
def get_model(nodes_in_hidden_layers, dropout, nr_of_inputs):
    model = Sequential()
    for i in range(len(nodes_in_hidden_layers)):
        if i == 0:
            model.add(Dense(nodes_in_hidden_layers[i], activation='swish', input_shape=(nr_of_inputs,)))  # Input and hidden layer
        else:
            if dropout != 0:
                model.add(Dropout(dropout))
            model.add(Dense(nodes_in_hidden_layers[i], activation='swish'))  # Additional hidden layers
    model.add(Dense(1, activation="linear"))  # Output layer
    model.compile(optimizer=Adam(learning_rate=0.001), loss="mean_squared_error", metrics=("mean_squared_error"))
    return model

In [3]:
def get_all_features(data):
    station = np.eye(19)[data["Base Station"]]
    year = np.eye(4)[data["Year"] - 2015]
    month = np.eye(12)[data["Month"] - 1]
    day = np.eye(31)[data["Day"] - 1]
    week = np.eye(53)[data["Week"] - 1]
    weekday = np.eye(7)[data["Weekday"] - 1]
    hour = np.eye(24)[data["Hour"]]
    daytime = np.zeros((len(data), 6))

    # Loop over the four-hour periods and sum the one hot encoded values of each hour in the period
    for i in range(6):
        daytime[:, i] = np.sum(hour[:, i*4:(i+1)*4], axis=1)

    x = np.concatenate([
        station,
        year, 
        month, 
        day, 
        week, 
        weekday,
        hour,
        daytime], axis=1)
    return x

In [4]:
def get_features(data):
    station = np.eye(19)[data["Base Station"]]
    year = np.eye(4)[data["Year"] - 2015]
    month = np.eye(12)[data["Month"] - 1]
    day = np.eye(31)[data["Day"] - 1]
    #week = np.eye(53)[data["Week"] - 1]
    weekday = np.eye(7)[data["Weekday"] - 1]
    hour = np.eye(24)[data["Hour"]]
    daytime = np.zeros((len(data), 6))

    # Loop over the four-hour periods and sum the one hot encoded values of each hour in the period
    for i in range(6):
        daytime[:, i] = np.sum(hour[:, i*4:(i+1)*4], axis=1)

    x = np.concatenate([
        station,
        year, 
        month, 
        day, 
        #week, 
        weekday,
        hour,
        daytime], axis=1)
    return x

In [5]:
def get_original_features(data):
    def get_daytime(hour, period_hours):
        periods = int(24/period_hours)
        for i in range(periods):
            start_hour = i
            end_hour = (i+1) * period_hours
            if start_hour <= hour < end_hour:
                return i
        return 6

    # Map the Hour column to the Daytime column using the map_hour_to_daytime function
    data["Daytime"] = data["Hour"].apply(lambda x: get_daytime(x, 4))

    return data[["Base Station", "Year", "Month", "Day", "Week", "Weekday", "Hour", "Daytime"]].values

In [6]:
def revert_all_features(data):
    base_station = np.argmax(data[:, :19], axis=1) 
    year = np.argmax(data[:, 19:23], axis=1) + 2015
    month = np.argmax(data[:, 23:35], axis=1) + 1
    day = np.argmax(data[:, 35:66], axis=1) + 1
    week = np.argmax(data[:, 66:119], axis=1) + 1
    weekday = np.argmax(data[:, 119:126], axis=1) + 1
    hour = np.argmax(data[:, 126:150], axis=1)
    daytime = np.argmax(data[:, 150:], axis=1)


    return list(zip(base_station, year, month, day, week, weekday, hour, daytime))

In [7]:
def revert_features(data):
    base_station = np.argmax(data[:, :19], axis=1) 
    year = np.argmax(data[:, 19:23], axis=1) + 2015
    month = np.argmax(data[:, 23:35], axis=1) + 1
    day = np.argmax(data[:, 35:66], axis=1) + 1
    #week = np.argmax(data[:, 66:119], axis=1) + 1
    weekday = np.argmax(data[:, 66:73], axis=1) + 1
    hour = np.argmax(data[:, 73:97], axis=1)
    daytime = np.argmax(data[:, 97:], axis=1)

    return list(zip(base_station, year, month, day, weekday, hour, daytime))

In [31]:
def preprocess(data, s):
    if s != None:
        data = data[data["Base Station"] == s]
    x = get_all_features(data)
    y = data["Incidents"].to_numpy()
    return x, y

In [32]:
DISTRIBUTION_FILE = "data/incidents_distribution_station_count.csv"
DISTRIBUTION_TEST_FILE = "data/incidents_distribution_station_truths.csv"

data = pd.read_csv(DISTRIBUTION_FILE, encoding='utf-8', escapechar='\\', parse_dates=True)
data_test = pd.read_csv(DISTRIBUTION_TEST_FILE, encoding='utf-8', escapechar='\\', parse_dates=True)

In [35]:
s = None
X_train, Y_train = preprocess(data, s)
X_test, Y_test = preprocess(data_test, s)

In [37]:
x, y = preprocess(data, None)
X_train, X_test, Y_train, Y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [ ]:
layers = [128, 32, 8]
model = get_model(layers, 0.4, len(X_train[0]))
es = EarlyStopping(monitor='mean_squared_error', mode='min', verbose=1, patience=1)
model.fit(X_train, Y_train, epochs=10, verbose=1, callbacks=[es])

In [33]:
predictions = []
Y_test = []
X_test = []

for s in range(0, 19):
    X_train, Y_train = preprocess(data, s)
    X_test_s, Y_test_s = preprocess(data_test, s)

    layers = [128, 64, 16]
    model = get_model(layers, 0.4, len(X_train[0]))
    es = EarlyStopping(monitor='val_mean_squared_error', mode='min', verbose=0, patience=5)
    model.fit(X_train, Y_train, epochs=20, verbose=0, validation_split=0.2, shuffle=False, callbacks=[es])

    """# Fit the Poisson regression model
    model = sm.GLM(Y_train, X_train, family=sm.families.Poisson())
    model = sm.GLM(Y_train, X_train, family=sm.families.NegativeBinomial())
    model = sm.ZeroInflatedGeneralizedPoisson(Y_train, X_train, exog_infl=X_train, inflation='logit')
    model = model.fit() """

    predictions_station = model.predict(X_test_s, verbose=0)
    print(mean_squared_error(Y_test_s, predictions_station))

    predictions.extend(predictions_station)
    X_test.extend(X_test_s)
    Y_test.extend(Y_test_s)

predictions = np.array(predictions)
Y_test = np.array(Y_test)
X_test = np.array(X_test)

0.23058850924523366
0.1533984212153746
0.3548880900815104
0.14265833502319383
0.6717456316902511
0.09450703926068733
0.5010540285827904
2.7282929605159913
0.9659905322206545
0.4581834194785659
0.3487048874219589
0.4534618868516402
0.2586836173780106
0.3545287888227806
0.49565986206001833
0.6639082620814708
0.5311237616931019
0.43739062205775103
0.37054288161634147


In [ ]:
# Fit the Poisson regression model
model = sm.GLM(Y_train, X_train, family=sm.families.Poisson())
model = model.fit()

# Print the model summary
print(model.summary())

In [ ]:
predictions = model.predict(X_test)

In [34]:
mse = mean_squared_error(Y_test, predictions)
print("Mean squared error: ", mse)

X_test_reverted = revert_all_features(X_test)

print("Base Station", "Year", "Month", "Day", "Week", "Weekday", "Hour", "Daytime")
for i in range(len(predictions)):
    print(predictions[i], np.round(predictions[i], 2), Y_test[i], X_test_reverted[i])

Mean squared error:  0.5376479756472278
Base Station Year Month Day Week Weekday Hour Daytime
[0.24009776] [0.24] 0 (0, 2017, 8, 6, 31, 7, 0, 0)
[0.25357988] [0.25] 0 (0, 2017, 8, 6, 31, 7, 1, 0)
[0.22643957] [0.23] 0 (0, 2017, 8, 6, 31, 7, 2, 0)
[0.21055561] [0.21] 0 (0, 2017, 8, 6, 31, 7, 3, 0)
[0.21361312] [0.21] 0 (0, 2017, 8, 6, 31, 7, 4, 1)
[0.22214293] [0.22] 0 (0, 2017, 8, 6, 31, 7, 5, 1)
[0.23762608] [0.24] 0 (0, 2017, 8, 6, 31, 7, 6, 1)
[0.24759904] [0.25] 0 (0, 2017, 8, 6, 31, 7, 7, 1)
[0.40398946] [0.4] 0 (0, 2017, 8, 6, 31, 7, 8, 2)
[0.42712426] [0.43] 0 (0, 2017, 8, 6, 31, 7, 9, 2)
[0.4896318] [0.49] 0 (0, 2017, 8, 6, 31, 7, 10, 2)
[0.43668106] [0.44] 0 (0, 2017, 8, 6, 31, 7, 11, 2)
[0.40722966] [0.41] 0 (0, 2017, 8, 6, 31, 7, 12, 3)
[0.48666188] [0.49] 0 (0, 2017, 8, 6, 31, 7, 13, 3)
[0.38956207] [0.39] 0 (0, 2017, 8, 6, 31, 7, 14, 3)
[0.38334665] [0.38] 0 (0, 2017, 8, 6, 31, 7, 15, 3)
[0.37643906] [0.38] 1 (0, 2017, 8, 6, 31, 7, 16, 4)
[0.3404281] [0.34] 0 (0, 2017, 8, 

In [55]:
PREDICTIONS_FILE = "data/incidents_distribution_station_predictions.json"

def save_predictions(predictions, X_test_reverted):
    d = {}
    for station in range(0, 19):
        d[station] = {}
        for day in range(6, 14):
            d[station][day] = {}
            for hour in range(0, 24):
                d[station][day][hour] = {}
    for prediction, x in zip(predictions, X_test_reverted):

        station = x[0]
        day = x[3]
        hour = x[6]
    
        d[station][day][hour] = round(float(prediction), 3)
    print("Saving predictions to file...")
    with open(PREDICTIONS_FILE, 'w') as f:
        json.dump(d, f, indent=2)


save_predictions(predictions, X_test_reverted)

Saving predictions to file...
